In [7]:
import pandas as pd
dataset = pd.read_csv('/Users/chris/OneDrive/Documentos/Asimov_Academy/AI_Agents/02_OpenAI_API/supermarket_sales.csv')

dataset.head(5)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [8]:
import openai
from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())

client = openai.Client()

In [9]:
import openai
print(openai.__version__)

1.95.1


### Criar o Assistant

In [10]:
file = client.files.create(
    file=open('/Users/chris/OneDrive/Documentos/Asimov_Academy/AI_Agents/02_OpenAI_API/supermarket_sales.csv','rb'),
    purpose='assistants'
)


In [11]:
file.id

'file-3BQNZ5SwF2bWBwamSafRtu'

In [13]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

assistant = client.beta.assistants.create(
    name="Analista Financeiro",
    instructions="Você é um analista financeiro de um supermercado. Você deve usar os dados  \
    .csv informados relativos às vendas do supermercado para realizar as suas análises",
    tools=[{"type": "code_interpreter"}],
    tool_resources={'code_interpreter': {'file_ids':[file.id]}},
    model="gpt-4-1106-preview"
)


### Criação de um Thread

In [14]:
thread = client.beta.threads.create()

C:\Users\chris\AppData\Local\Temp\ipykernel_11892\3600338518.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()


### Adicionar mensagem à thread criada

In [26]:
texto_mensagem='Qual é o rating médio das vendas do supermercado?'
texto_mensagem='Gere um gráfico de Pizza com o percentual dos meios de pagamento. O arquivo está em formato.csv'

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content=texto_mensagem)

C:\Users\chris\AppData\Local\Temp\ipykernel_11892\877700454.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


### Rodar a Thread no assistant

In [27]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='O nome do usuário é Usuário 9999'
)

C:\Users\chris\AppData\Local\Temp\ipykernel_11892\195674185.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


### Aguardar a Thread rodar

In [28]:

import time


while run.status in ['queued','in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )


C:\Users\chris\AppData\Local\Temp\ipykernel_11892\641565305.py:6: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [29]:
run.status

'completed'

### Verificar a Resposta

In [30]:
if run.status =='completed':
    messages= client.beta.threads.messages.list(
        thread_id=thread.id
    )

    print(messages)

else:
    print('Erro', run.status)

C:\Users\chris\AppData\Local\Temp\ipykernel_11892\1184797989.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages= client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_bi5WL1r69GDl9wCOdPwqgsFU', assistant_id='asst_AZjFVejnqkbbbxKLNvfDaRTY', attachments=[Attachment(file_id='file-Niag6ufdLtScadqgEpg5hd', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-2GVLJMgoX5mYXQHhAGoN4t', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=275, file_path=FilePath(file_id='file-Niag6ufdLtScadqgEpg5hd'), start_index=211, text='sandbox:/mnt/data/supermarket_payment_distribution_pie_chart.png', type='file_path')], value='O gráfico de pizza com o percentual dos meios de pagamento foi gerado e salvo com sucesso. Você pode baixar o gráfico diretamente através do seguinte link:\n\n[Download do gráfico de pizza dos meios de pagamento](sandbox:/mnt/data/supermarket_payment_distribution_pie_chart.png)'), type='text')], created_at=1753147818, incomplete_at=None, incomplete_details=None, met

In [32]:
print(messages.data[0].content[0])

ImageFileContentBlock(image_file=ImageFile(file_id='file-2GVLJMgoX5mYXQHhAGoN4t', detail=None), type='image_file')


### Analisando os passos do modelo

In [33]:
run_steps= client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
    
)




C:\Users\chris\AppData\Local\Temp\ipykernel_11892\687884520.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps= client.beta.threads.runs.steps.list(


In [34]:
run_steps.data[0]

RunStep(id='step_w2jyWBoTmIN5WwVJd8nnU68q', assistant_id='asst_AZjFVejnqkbbbxKLNvfDaRTY', cancelled_at=None, completed_at=1753147820, created_at=1753147818, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_osYwubScjayAO3mQZDaC6kll', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_bi5WL1r69GDl9wCOdPwqgsFU'), type='message_creation'), thread_id='thread_0I8iIMLnba7XSu460UGHsZsU', type='message_creation', usage=Usage(completion_tokens=68, prompt_tokens=1637, total_tokens=1705, prompt_token_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0}), expires_at=None)

In [44]:
for step in run_steps.data:
    print("=== Step:", step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print("-----------")
            print(tool_call.code_interpreter.input)
            print("-----------")
            print("Result")
            if tool_call.code_interpreter.outputs[0].type== 'logs':
                print(tool_call.code_interpreter.outputs[0].logs)
            if step.step_details.type == 'message_creation':
                message = client.beta.threads.messages.retrieve(
                thread_id=thread.id,
                message_id=step.step_details.message_creation.message_id
                )
            if message.content[0].type == 'text':
                print(message.content[0].text.value)

            if message.content[0].type == 'image_file':
                file_id = message.content[0].image_file.file_id
                image_data = client.files.content(file_id)
                with open(f'arquivos/{file_id}.png', 'wb') as f:
                    f.write(image_data.read())
                    print(f'Imagem {file_id} salva com sucesso!')
                import matplotlib.pyplot as plt
                import matplotlib.image as mpimg

                img = mpimg.imread(f'arquivos/{file_id}.png')
                fig, ax = plt.subplots()
                ax.set_axis_off()
                ax.imshow(img)
                plt.show()

            

=== Step: message_creation
=== Step: tool_calls
-----------
import matplotlib.pyplot as plt

# Calcular a contagem de cada meio de pagamento
payment_counts = df['Payment'].value_counts()

# Calcular as porcentagens para cada meio de pagamento
payment_percentage = payment_counts / payment_counts.sum() * 100

# Criar o gráfico de pizza
plt.figure(figsize=(8, 8))
plt.pie(payment_percentage, labels=payment_percentage.index, autopct='%1.1f%%', startangle=140)
plt.title('Percentual dos Meios de Pagamento')
plt.axis('equal')  # Para garantir que o gráfico de pizza seja um círculo

# Salvar a figura
output_file_path = '/mnt/data/supermarket_payment_distribution_pie_chart.png'
plt.savefig(output_file_path)

# Mostrar o gráfico
plt.show()

# Retornar o caminho do arquivo onde o gráfico foi salvo
output_file_path
-----------
Result


FileNotFoundError: [Errno 2] No such file or directory: 'arquivos/file-2GVLJMgoX5mYXQHhAGoN4t.png'